In [169]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)


In [234]:
main = pd.read_csv('main2.csv')

C:\Users\HP\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [400]:
main.shape

(1642511, 48)

In [235]:
main['FirstRegistiration']= pd.to_datetime(main['FirstRegistiration'])
main['InvoiceDate']= pd.to_datetime(main['InvoiceDate'])

# Creating 6 months splits

In [236]:
import datetime

def get_last_n_days(date):
    days = [date - datetime.timedelta(x) for x in range(180)]
    return days


In [237]:
label = main[main.IsMalfunction == 1][['VehicleId', 'InvoiceDate']].drop_duplicates().reset_index().drop('index', axis=1)

In [238]:
label['Days'] = label.InvoiceDate.apply(get_last_n_days)

In [239]:
label_exploded = label.explode('Days').reset_index().drop('index', axis = 1)

In [240]:
label_exploded['InvoiceDate'] = 1
label_exploded.rename(columns={'InvoiceDate': 'label'}, inplace=True)
label_exploded.rename(columns={'Days': 'InvoiceDate'}, inplace=True)

In [241]:
main = pd.merge(main, label_exploded, how='left', on=['VehicleId', 'InvoiceDate'])

In [242]:
main.label.fillna(0, inplace=True)

In [243]:
main.drop('next_six',axis = 1,inplace=True)

# Feature Extreaction

In [244]:
main['InvoiceDate'] = main['InvoiceDate'].astype(str)
main['Id'] = main['InvoiceDate'] + main['VehicleId']
q1 = main.groupby('Id')['ModelYear'].count()
q2 = main[main.IsMalfunction == 1].groupby(['Id'])['ModelYear'].count()
main = pd.merge(main,q1,how = 'left',on=['Id'])
main = pd.merge(main,q2,how = 'left',on=['Id'])

In [245]:
main.rename(columns = {'ModelYear_y':'ismal_one_for_ıd','ModelYear':'ismal_for_ıd'},inplace=True)

In [246]:
main['ismal_for_ıd'].fillna(0,inplace = True)

In [247]:
main['for_cumsum_zero_ismal'] = np.where((main.IsMalfunction == 0),1,0)

In [248]:
main['for_cumsum_one_ismal'] = np.where((main.IsMalfunction == 1),1,0)

In [249]:
main['total_zero_for_vehicle'] = main.groupby('VehicleId')['for_cumsum_zero_ismal'].cumsum()

In [250]:
main['total_one_for_vehicle'] = main.groupby('VehicleId')['for_cumsum_one_ismal'].cumsum()

In [251]:
main['InvoiceDate'] = pd.to_datetime(main['InvoiceDate'])
aa=main.groupby('VehicleId')['InvoiceDate'].diff().dt.days


In [252]:
bb = main.groupby('VehicleId')['CustomerId'].apply(lambda x: (~pd.Series(x).duplicated()).cumsum())
main['cum_uni_customer_numuber'] = bb
cc = main.groupby('BranchCode')['IsMalfunction'].expanding().mean()
cc_index  = []

for i in range(cc.shape[0]):
    cc_index.append(cc.index[i][1])

cc.index = cc_index
main['cum_mean_for_branch_ismal'] = cc

In [253]:
dd = main.groupby('BodyCode')['IsMalfunction'].expanding().mean()
dd_index  = []

for i in range(dd.shape[0]):
    dd_index.append(dd.index[i][1])

dd.index = dd_index
main['cum_mean_for_bodycode_ismal'] = dd

ee = main.groupby('City')['IsMalfunction'].expanding().mean()

ee_index  = []

for i in range(ee.shape[0]):
    ee_index.append(ee.index[i][1])

ee.index = ee_index
main['cum_mean_for_city_ismal'] = ee

In [192]:
xx = main.groupby('CustomerId')['VehicleId'].nunique()
yy = main.groupby('CustomerId')['IsMalfunction'].mean()
xx = pd.DataFrame(xx)
yy = pd.DataFrame(yy)
xx.rename(columns= {'VehicleId':'uni_customer'},inplace = True)
yy.rename(columns= {'VehicleId':'customer_ismal_uni'},inplace = True)
xxyy = pd.merge(xx,yy,how= 'left',on=['CustomerId'])

from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler
X= xxyy
sc = MinMaxScaler()
X = sc.fit_transform(X)
kmeans = KMeans(n_clusters=5, random_state=0).fit(X)
b = kmeans.labels_

a = kmeans.predict(X)

c = kmeans.cluster_centers_
xxyy['cluster_for_customer'] = b
xxyy = xxyy[['cluster_for_customer']]
main = pd.merge(main,xxyy,how = 'left',on = 'CustomerId')

In [193]:
ff = main.groupby('Serie')['IsMalfunction'].mean()
gg = main.groupby('Serie')['ModelYear_x'].count()
ff = pd.DataFrame(ff)
gg = pd.DataFrame(gg)

ffgg = pd.merge(ff,gg,how= 'left',on=['Serie'])

In [194]:


from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler
X= ffgg[['IsMalfunction','ModelYear_x']]
sc = MinMaxScaler()
X = sc.fit_transform(X)
kmeans = KMeans(n_clusters=5, random_state=0).fit(X)
b = kmeans.labels_

a = kmeans.predict(X)

c = kmeans.cluster_centers_
ffgg['cluster_for_serie'] = b

ffgg = ffgg[['cluster_for_serie']]
main = pd.merge(main,ffgg,how = 'left',on = ['Serie'])

In [195]:
ll = main.groupby('SplitCode')['IsMalfunction'].mean()
mm = main.groupby('SplitCode')['ModelYear_x'].count()
ll = pd.DataFrame(ll)
mm = pd.DataFrame(mm)

llmm = pd.merge(ll,mm,how= 'left',on=['SplitCode'])

from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler
X= llmm[['IsMalfunction','ModelYear_x']]
sc = MinMaxScaler()
X = sc.fit_transform(X)
kmeans = KMeans(n_clusters=7, random_state=0).fit(X)
b = kmeans.labels_

a = kmeans.predict(X)

c = kmeans.cluster_centers_
llmm['cluster_for_splitcode'] = b

llmm = llmm[['cluster_for_splitcode']]
main = pd.merge(main,llmm,how = 'left',on = ['SplitCode'])

In [196]:
oo = main.groupby('ServiceInvoiceLineNo')['IsMalfunction'].mean()
pp = main.groupby('ServiceInvoiceLineNo')['ModelYear_x'].count()
oo = pd.DataFrame(oo)
pp = pd.DataFrame(pp)

oopp = pd.merge(oo,pp,how= 'left',on=['ServiceInvoiceLineNo'])

from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler
X= oopp[['IsMalfunction','ModelYear_x']]
sc = MinMaxScaler()
X = sc.fit_transform(X)
kmeans = KMeans(n_clusters=7, random_state=0).fit(X)
b = kmeans.labels_

a = kmeans.predict(X)

c = kmeans.cluster_centers_
oopp['cluster_for_ServiceInvoiceLineNo'] = b

oopp = oopp[['cluster_for_ServiceInvoiceLineNo']]
main = pd.merge(main,oopp,how = 'left',on = ['ServiceInvoiceLineNo'])

In [254]:
main['car_year'] = main.InvoiceDate.dt.year
main['how_many_years'] = main.car_year - main.ModelYear_x
main.drop('car_year',axis = 1,inplace=True)

In [255]:
main['how_may_days'] = (main.InvoiceDate - main.FirstRegistiration).dt.days

In [402]:
main.shape

(1642511, 48)

In [256]:
ee = main.groupby('Make')['IsMalfunction'].expanding().mean()

ee_index  = []

for i in range(ee.shape[0]):
    ee_index.append(ee.index[i][1])

ee.index = ee_index
main['cum_mean_for_Make_ismal'] = ee

In [257]:
ee = main.groupby('ModelYear_x')['IsMalfunction'].expanding().mean()

ee_index  = []

for i in range(ee.shape[0]):
    ee_index.append(ee.index[i][1])

ee.index = ee_index
main['cum_mean_for_ModelYear_x_ismal'] = ee

In [258]:
ee = main.groupby('ModelCode')['IsMalfunction'].expanding().mean()

ee_index  = []

for i in range(ee.shape[0]):
    ee_index.append(ee.index[i][1])

ee.index = ee_index
main['cum_mean_for_ModelCode_ismal'] = ee

In [259]:
ee = main.groupby('SubGroupDescription')['IsMalfunction'].expanding().mean()

ee_index  = []

for i in range(ee.shape[0]):
    ee_index.append(ee.index[i][1])

ee.index = ee_index
main['cum_mean_for_SubGroupDescription_ismal'] = ee

In [260]:
ee = main.groupby('MainGroupDescription')['IsMalfunction'].expanding().mean()

ee_index  = []

for i in range(ee.shape[0]):
    ee_index.append(ee.index[i][1])

ee.index = ee_index
main['cum_mean_for_MainGroupDescription_ismal'] = ee

In [261]:
ee = main.groupby('ServiceInvoiceLineNo')['IsMalfunction'].expanding().mean()

ee_index  = []

for i in range(ee.shape[0]):
    ee_index.append(ee.index[i][1])

ee.index = ee_index
main['cum_mean_for_ServiceInvoiceLineNo_ismal'] = ee

In [1]:
feature_list = ['IsMalfunction_x', 'label', 'ismal_one_for_ıd', 'ismal_for_ıd',
       'for_cumsum_zero_ismal', 'for_cumsum_one_ismal',
       'total_zero_for_vehicle', 'total_one_for_vehicle', 'day_diff',
       'cum_uni_customer_numuber', 'cum_mean_for_branch_ismal',
       'cum_mean_for_bodycode_ismal', 'cum_mean_for_city_ismal',
       'uni_customer', 'IsMalfunction_y', 'cluster_for_customer',
       'cluster_for_serie', 'cluster_for_splitcode', 'is_dahili',
       'how_may_days']

In [263]:
ee = main.groupby('ItemDescription')['IsMalfunction'].expanding().mean()

ee_index  = []

for i in range(ee.shape[0]):
    ee_index.append(ee.index[i][1])

ee.index = ee_index
main['cum_mean_for_ItemDescription_ismal'] = ee

In [208]:
for i in main.columns:
    if 'cluster' in i:
        print(i)

cluster_for_customer
cluster_for_serie
cluster_for_splitcode
cluster_for_ServiceInvoiceLineNo


In [209]:
new_features = ['Make','BodyCode','ModelCode','BranchCode','MainGroupDescription','SubGroupDescription','Split_Grubu',
'ItemDescription','ItemNo','SubGroupCode']

In [210]:
for i in new_features:
    if main[i].nunique() < 5:
        n = main[i].nunique()
    elif main[i].nunique() >= 5:
        n = 5
    oo = main.groupby(i)['IsMalfunction'].mean()
    pp = main.groupby(i)['ModelYear_x'].count()
    oo = pd.DataFrame(oo)
    pp = pd.DataFrame(pp)

    oopp = pd.merge(oo,pp,how= 'left',on=[i])

    from sklearn.cluster import KMeans
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler
    X= oopp[['IsMalfunction','ModelYear_x']]
    sc = MinMaxScaler()
    X = sc.fit_transform(X)
    kmeans = KMeans(n_clusters=n, random_state=0).fit(X)
    b = kmeans.labels_

    a = kmeans.predict(X)

    c = kmeans.cluster_centers_
    oopp['cluster_for_'+i] = b

    oopp = oopp[['cluster_for_'+i]]
    main = pd.merge(main,oopp,how = 'left',on = [i])

In [211]:
cluster_features = [i for i in main.columns if 'cluster' in i]

In [289]:
main.head()

,Unnamed: 0,index,City,CustomerId,VehicleId,FirstRegistiration,Make,BodyCode,Serie,ModelCode,ModelYear_x,InvoiceDate,ServiceInvoiceDate,BranchCode,Milage,MainGroupDescription,SubGroupDescription,SplitCode,Split_Grubu,ServiceInvoiceLineNo,ItemDescription,ItemNo,ItemType,MainGroupCode,SubGroupCode,IsMalfunction,label,Id,ismal_one_for_ıd,ismal_for_ıd,for_cumsum_zero_ismal,for_cumsum_one_ismal,total_zero_for_vehicle,total_one_for_vehicle,day_diff,cum_uni_customer_numuber,cum_mean_for_branch_ismal,cum_mean_for_bodycode_ismal,cum_mean_for_city_ismal,how_many_years,how_may_days,cum_mean_for_Make_ismal,cum_mean_for_ModelYear_x_ismal,cum_mean_for_ModelCode_ismal,cum_mean_for_SubGroupDescription_ismal,cum_mean_for_MainGroupDescription_ismal,cum_mean_for_ServiceInvoiceLineNo_ismal,cum_mean_for_ItemDescription_ismal
0,0,694367,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,20000,Teslim Öncesi Bakım,0000009,İşçilik,01,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,1,0,NaN,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,803348,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,130000,Bmw & Bmw-Mını Oto Yıkama,75,Harici İşçilik,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,2,0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,957140,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,80000,TRAFİK SETİ,99716000001,Parça,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,3,0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1096269,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,90000,YANGIN SÖNDÜRÜCÜ,99726000001,Parça,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,4,0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1392109,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,70000,EURO DIZEL 1,99160000666,Parça,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,5,0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
main2 = main

In [294]:
main = main2

In [398]:
main.to_csv('main3.csv')

In [379]:
test = main[(main.InvoiceDate < pd.to_datetime('01.04.2021'))&(main.InvoiceDate > pd.to_datetime('01.10.2020'))]

In [380]:
train = main[main.InvoiceDate < pd.to_datetime('01.10.2020')]

In [381]:
feature_list = list(main.columns[25:])
train = train[feature_list]
train.shape
test = test[feature_list]
test.shape

(366458, 23)

# Train Test Split and Training

In [382]:
train = pd.DataFrame(train.groupby(['Id'], sort = False).max())

In [383]:
test = pd.DataFrame(test.groupby(['Id'], sort = False).max())

In [384]:
X_train = train.drop('label',axis = 1)
X_test = test.drop('label',axis = 1)
y_train = train['label']
y_test = test['label']

In [396]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(verbose = False)
model.fit(X_train,y_train)
predictions = model.predict(X_test)

In [393]:
from sklearn.metrics import f1_score
score = f1_score(y_test,predictions)
print(score)


0.70800537531196


In [394]:
from sklearn.metrics import f1_score
score = f1_score(y_train,predictions_train)
print(score)

0.7743732590529249


In [309]:
test['Id'] = main.iloc[566827:933285,27]

In [318]:
X_test.head()

,Id,label,pred
Id,,,
2020-01-1102i0Y000004frKaQAI,NaN,0.0,0.0
2020-01-1102i0Y000004fts9QAA,NaN,0.0,0.0
2020-01-1102i0Y000004fwsgQAA,NaN,0.0,0.0
2020-01-1102i0Y000004g6vjQAA,NaN,0.0,0.0
2020-01-1102i0Y000004hE0DQAU,NaN,0.0,0.0


In [275]:
test.tail()

,IsMalfunction,label,ismal_one_for_ıd,ismal_for_ıd,for_cumsum_zero_ismal,for_cumsum_one_ismal,total_zero_for_vehicle,total_one_for_vehicle,day_diff,cum_uni_customer_numuber,cum_mean_for_branch_ismal,cum_mean_for_bodycode_ismal,cum_mean_for_city_ismal,how_many_years,how_may_days,cum_mean_for_Make_ismal,cum_mean_for_ModelYear_x_ismal,cum_mean_for_ModelCode_ismal,cum_mean_for_SubGroupDescription_ismal,cum_mean_for_MainGroupDescription_ismal,cum_mean_for_ServiceInvoiceLineNo_ismal,cum_mean_for_ItemDescription_ismal
933280,0,0.0,2,0.0,1,0,1,0,NaN,1,0.007192,0.000047,0.007479,0,6.0,0.007531,0.000390,0.000047,0.012127,0.013034,0.009782,0.0
933281,0,0.0,2,0.0,1,0,2,0,0.0,1,0.007192,0.000047,0.007479,0,6.0,0.007531,0.000390,0.000047,0.012127,0.013034,0.048748,0.0
933282,0,0.0,2,0.0,1,0,104,0,1.0,3,0.006946,0.000099,0.006421,1,497.0,0.007531,0.008341,0.000000,0.000000,0.000000,0.009782,0.0
933283,0,0.0,2,0.0,1,0,105,0,0.0,3,0.006945,0.000099,0.006421,1,497.0,0.007531,0.008340,0.000000,0.012127,0.013034,0.048747,0.0
933284,0,0.0,1,0.0,1,0,1,0,NaN,1,0.012875,0.015862,0.007479,1,3.0,0.018581,0.000390,0.001568,0.012127,0.013034,0.009782,0.0


In [310]:
test['pred'] = predictions

In [311]:
X_test = test[['Id','label','pred']]

In [321]:
X_test.drop('Id',axis = 1,inplace=True)
X_test.reset_index(inplace = True)

In [322]:
new_test = pd.DataFrame(X_test.groupby(['Id'], sort = False).max())

In [323]:
veh_list = []
for i in new_test.reset_index()['Id']:
    veh_list.append(i[10:])


In [324]:
new_test['VehicleId'] = veh_list

In [325]:
new_test = pd.DataFrame(new_test.groupby('VehicleId', sort = False).max()).reset_index()[['VehicleId','label','pred']]

In [326]:
new_test.head()

,VehicleId,label,pred
0,02i0Y000004frKaQAI,0.0,0.0
1,02i0Y000004fts9QAA,0.0,0.0
2,02i0Y000004fwsgQAA,0.0,0.0
3,02i0Y000004g6vjQAA,0.0,0.0
4,02i0Y000004hE0DQAU,0.0,0.0


In [327]:
from sklearn.metrics import f1_score
score = f1_score(new_test['label'],new_test['pred'])
print(score)


0.8897266729500471


In [160]:
new_test.head()

,VehicleId,real,pred
0,02i0Y000004fs2mQAA,0.0,0.0
1,02i0Y000004g8D9QAI,0.0,0.0
2,02i0Y000004h5QJQAY,0.0,0.0
3,02i0Y000004h6aOQAQ,1.0,1.0
4,02i0Y000004hDLPQA2,0.0,0.0


# Submission DF

In [92]:
sample_submission = pd.read_csv('sample_submission.csv')


In [93]:
sample_submission = pd.merge(sample_submission,new_test,how = 'left',on=['VehicleId'])

In [94]:
sample_submission.drop(['IsMalfunction','real'],axis = 1,inplace=True)
sample_submission.rename(columns={'pred':'IsMalfunction'},inplace=True)
sample_submission.fillna(0,inplace=True)
sample_submission.IsMalfunction.astype(int)

0        0
1        1
2        0
3        0
4        1
        ..
52419    0
52420    0
52421    0
52422    0
52423    0
Name: IsMalfunction, Length: 52424, dtype: int32

In [95]:
sample_submission.IsMalfunction = sample_submission.IsMalfunction.astype(int)

In [96]:
sample_submission.head()

,VehicleId,IsMalfunction
0,02i1i000007CEf9AAG,0
1,02i0Y000005Htw6QAC,1
2,02i1i000007CkokAAC,0
3,02i0Y000007Xft3QAC,0
4,02i1i00000EVPUbAAP,1


In [97]:
sample_submission = sample_submission.set_index('VehicleId')

In [98]:
sample_submission.to_csv('sub10.csv')

,Unnamed: 0,index,City,CustomerId,VehicleId,FirstRegistiration,Make,BodyCode,Serie,ModelCode,ModelYear_x,InvoiceDate,ServiceInvoiceDate,BranchCode,Milage,MainGroupDescription,SubGroupDescription,SplitCode,Split_Grubu,ServiceInvoiceLineNo,ItemDescription,ItemNo,ItemType,MainGroupCode,SubGroupCode,IsMalfunction,label,Id,ismal_one_for_ıd,ismal_for_ıd,for_cumsum_zero_ismal,for_cumsum_one_ismal,total_zero_for_vehicle,total_one_for_vehicle,day_diff,cum_uni_customer_numuber,cum_mean_for_branch_ismal,cum_mean_for_bodycode_ismal,cum_mean_for_city_ismal,cluster_for_customer,cluster_for_serie,cluster_for_splitcode,cluster_for_ServiceInvoiceLineNo,how_many_years,how_may_days,cum_mean_for_Make_ismal,cum_mean_for_ModelYear_x_ismal,cum_mean_for_ModelCode_ismal,cum_mean_for_SubGroupDescription_ismal,cum_mean_for_MainGroupDescription_ismal,cum_mean_for_ServiceInvoiceLineNo_ismal,cum_mean_for_ItemDescription_ismal,cluster_for_Make_x,cluster_for_BodyCode_x,cluster_for_ModelCode_x,cluster_for_Make_y,cluster_for_BodyCode_y,cluster_for_ModelCode_y,cluster_for_BranchCode,cluster_for_MainGroupDescription,cluster_for_SubGroupDescription,cluster_for_Split_Grubu,cluster_for_ItemDescription,cluster_for_ItemNo,cluster_for_SubGroupCode
0,0,694367,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,20000,Teslim Öncesi Bakım,0000009,İşçilik,01,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,1,0,NaN,1,0.0,0.0,0.0,2.0,3.0,3,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,3,1,1,3,1,2,2,1,3.0,4.0,1
1,1,803348,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,130000,Bmw & Bmw-Mını Oto Yıkama,75,Harici İşçilik,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,2,0,0.0,1,0.0,0.0,0.0,2.0,3.0,3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,3,1,1,3,1,2,2,1,3.0,4.0,1
2,2,957140,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,80000,TRAFİK SETİ,99716000001,Parça,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,3,0,0.0,1,0.0,0.0,0.0,2.0,3.0,3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,3,1,1,3,1,2,2,1,3.0,0.0,1
3,3,1096269,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,90000,YANGIN SÖNDÜRÜCÜ,99726000001,Parça,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,4,0,0.0,1,0.0,0.0,0.0,2.0,3.0,3,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,3,1,1,3,1,2,2,1,0.0,0.0,1
4,4,1392109,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,70000,EURO DIZEL 1,99160000666,Parça,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,5,0,0.0,1,0.0,0.0,0.0,2.0,3.0,3,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,3,1,1,3,1,2,2,1,0.0,0.0,1


In [399]:
main.head()

,Unnamed: 0,index,City,CustomerId,VehicleId,FirstRegistiration,Make,BodyCode,Serie,ModelCode,ModelYear_x,InvoiceDate,ServiceInvoiceDate,BranchCode,Milage,MainGroupDescription,SubGroupDescription,SplitCode,Split_Grubu,ServiceInvoiceLineNo,ItemDescription,ItemNo,ItemType,MainGroupCode,SubGroupCode,IsMalfunction,label,Id,ismal_one_for_ıd,ismal_for_ıd,for_cumsum_zero_ismal,for_cumsum_one_ismal,total_zero_for_vehicle,total_one_for_vehicle,day_diff,cum_uni_customer_numuber,cum_mean_for_branch_ismal,cum_mean_for_bodycode_ismal,cum_mean_for_city_ismal,how_many_years,how_may_days,cum_mean_for_Make_ismal,cum_mean_for_ModelYear_x_ismal,cum_mean_for_ModelCode_ismal,cum_mean_for_SubGroupDescription_ismal,cum_mean_for_MainGroupDescription_ismal,cum_mean_for_ServiceInvoiceLineNo_ismal,cum_mean_for_ItemDescription_ismal
0,0,694367,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,20000,Teslim Öncesi Bakım,0000009,İşçilik,01,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,1,0,NaN,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,803348,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,130000,Bmw & Bmw-Mını Oto Yıkama,75,Harici İşçilik,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,2,0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,957140,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,80000,TRAFİK SETİ,99716000001,Parça,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,3,0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,1096269,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,90000,YANGIN SÖNDÜRÜCÜ,99726000001,Parça,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,4,0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1392109,İstanbul,1157429,02i0Y000004hV14QAE,2015-03-02,MINI,F55,MINI Hatch,XT31,2015,2015-03-02,SINP023726,134533,0,Belirsiz,Belirsiz,5,other,70000,EURO DIZEL 1,99160000666,Parça,Belirsiz,Belirsiz,0,0.0,2015-03-0202i0Y000004hV14QAE,6,0.0,1,0,5,0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
